# Create Amazon SageMaker Autopilot job for pose classification
- Use Python 3 Data Scientist kernel in SageMaker Studio with ml.t3.medium instance
- Create a CSV file with pose keypoints output from PoseKeypointExtractionModel  
- Upload CSV file into S3 and run following code to generate an Autopilot job

In [ ]:
import boto3
from sagemaker import get_execution_role

client = boto3.client('sagemaker')
role = get_execution_role()
#print(role)

response = client.create_auto_ml_job(
    AutoMLJobName='Pose_Classification_Autopilot',
    InputDataConfig=[
        {
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': 's3://S3_BUCKET_NAME/PATH/TO/DATA.csv'
                }
            },
            'CompressionType': 'None',
            'TargetAttributeName': 'POSE_CLASS'
        },
    ],
    OutputDataConfig={
        'S3OutputPath': 's3://S3_BUCKET_NAME/AutoML_output/'
    },
    ProblemType='BinaryClassification',
    AutoMLJobObjective={
        'MetricName':'F1'
    },
    AutoMLJobConfig={
        'CompletionCriteria': {
            'MaxCandidates': 250,
            'MaxRuntimePerTrainingJobInSeconds': 86400,
            'MaxAutoMLJobRuntimeInSeconds': 2592000
        },
    },
    RoleArn = role,
    ModelDeployConfig={
        'AutoGenerateEndpointName': False,
        'EndpointName': 'Pose_Classification_Endpoint'
    }
)

In [ ]:
response = client.describe_auto_ml_job(
    AutoMLJobName='Pose_Classification_Autopilot'
)
response

# Host the model with pre-trained SageMaker model artifacts
- If you already have the pre-trained SageMaker model artifacts, upload them into S3 and run following cells to host the model

In [ ]:
model_name = 'Pose-Classification-Model'
s3_bucket = 'S3_BUCKET_NAME'
model_sklearn = 'MODEL_SKLEARN_KEY_PREFIX'
model_xgboost = 'MODEL_XGBOOST_KEY_PREFIX'

import boto3
from sagemaker import get_execution_role

client = boto3.client('sagemaker')
role = get_execution_role()
#print(role)

response = client.create_model(
    ModelName= model_name,
    Containers = [
        {
            "Image": "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.2.1-1-cpu-py3",
            "Mode": "SingleModel",
            "ModelDataUrl":  "s3://{}/{}/model.tar.gz".format(s3_bucket, model_sklearn),
            "Environment":  {
                "AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF": "1",
                "AUTOML_TRANSFORM_MODE": "feature-transform",
                "SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT": "application/x-recordio-protobuf",
                "SAGEMAKER_PROGRAM": "sagemaker_serve",
                "SAGEMAKER_SUBMIT_DIRECTORY": "/opt/ml/model/code"
            }
        },
        {
            "Image": "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.2-2-cpu-py3",
            "Mode": "SingleModel",
            "ModelDataUrl": "s3://{}/{}/model.tar.gz".format(s3_bucket, model_xgboost),
            "Environment": {
                "MAX_CONTENT_LENGTH": "20971520",
                "SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT": "text/csv",
                "SAGEMAKER_INFERENCE_INPUT": "predicted_label,probability",
                "SAGEMAKER_INFERENCE_OUTPUT": "predicted_label,probability",
                "SAGEMAKER_INFERENCE_SUPPORTED": "predicted_label,probability,probabilities"
            }
        },
        {
            "Image": "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.2.1-1-cpu-py3",
            "Mode": "SingleModel",
            "ModelDataUrl": "s3://{}/{}/model.tar.gz".format(s3_bucket, model_sklearn),
            "Environment": {
                "AUTOML_TRANSFORM_MODE": "inverse-label-transform",
                "SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT": "text/csv",
                "SAGEMAKER_INFERENCE_INPUT": "predicted_label,probability",
                "SAGEMAKER_INFERENCE_OUTPUT": "predicted_label,probability",
                "SAGEMAKER_INFERENCE_SUPPORTED": "predicted_label,probability,labels,probabilities",
                "SAGEMAKER_PROGRAM": "sagemaker_serve",
                "SAGEMAKER_SUBMIT_DIRECTORY": "/opt/ml/model/code"
            }
        }
    ],
    InferenceExecutionConfig =  {
        "Mode": "Serial"
    },
    ExecutionRoleArn = role,    
)

In [ ]:
from time import gmtime, strftime
endpoint_config_name = "Pose-Classification-EndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint config name: " + endpoint_config_name)

create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 2,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)


In [ ]:
endpoint_name = "Pose-ClassificationEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Endpoint name: " + endpoint_name)

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Endpoint Status: " + status)

print("Waiting for {} endpoint to be in service...".format(endpoint_name))
waiter = client.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=endpoint_name)

In [ ]:
# Test with SageMaker runtime client
rt_client = boto3.client('sagemaker-runtime')
# class 1
payload = '0.5,0.203,0.458,0.266,0.688,0.281,0.104,0.203,0.708,0.453,0.458,0.516,0.604,0.516,0.292,0.812,0.833,0.812'
# class 0
#payload = '0.562,0.25,0.667,0.328,0.479,0.297,0.771,0.547,0.396,0.516,0.646,0.547,0.5,0.562,0.688,0.75,0.604,0.766'
response = rt_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType='text/csv')

result = response["Body"].read().decode("utf-8")
scene = int(result.split(',')[0])
conf = float(result.split(',')[1])
print(scene,conf)

# Delete the endpoint after use

In [ ]:
client.delete_endpoint(EndpointName=endpoint_name)